In [10]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql duckdb:///:memory:
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

,Count
0,14


In [12]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,COALESCE(results.Stage_Date, races.RaceStart) AS Stage_Date
    ,results.Stage_Type
    ,CASE WHEN results.Stage_ID <> 'gc' AND (results.Stage_Type <> 'TTT' OR results.Stage_Type IS NULL) THEN 1 ELSE 0 END AS Løbsdage
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,riders.RiderPrice
    ,riders.RiderTeam
    ,CASE WHEN cbans.RiderName IS NULL AND riders.RiderPrice <= 2.5 THEN 'Ja' ELSE '' END AS Cheapo
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Riders riders
LEFT JOIN Results_2025 results ON results.Rider = riders.RiderName_PCS
LEFT JOIN Races races ON races.RaceName_PCS = results.Race
LEFT JOIN CheapoBans cbans ON cbans.RiderName = riders.RiderName_Zweeler
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_ID LIKE '%stage%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

,Count
0,7399


In [13]:
%%sql
rider_output << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeams
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(Points) AS int), 0) AS Point
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src;

In [14]:
%%sql
rider_output_cheapo << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeamsCheapo
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(CheapoPoints) AS int), 0) AS Point
        ,MAX(Cheapo) AS Cheapo
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     Navn
    ,Hold
    ,Pris
    ,Løbsdage
    ,Point
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src
WHERE Cheapo = 'Ja';

In [43]:
%%sql
WITH Src AS
(
    SELECT YEARWEEK(results.Stage_Date) AS Uge, teams.ManagerName AS Manager, CAST(SUM(Points) AS int) AS Point
    FROM RaceResultsPoints results
    INNER JOIN ManagerTeams teams ON teams.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND Uge IS NOT NULL
    GROUP BY YEARWEEK(results.Stage_Date), teams.ManagerName
)
PIVOT Src
ON Uge
USING COALESCE(SUM(Point)::int, 0)
GROUP BY Manager
ORDER BY Manager;

,Manager,202504,202505,202506,202507,202508,202509,202510,202511
0,Chrelle,312,100,67,130,653,72,280,36
1,Hustlersen,484,26,6,188,437,0,161,36
2,Jappo,156,130,15,130,599,139,379,71
3,Jarma,8,105,29,38,557,179,207,55
4,Kenk,283,49,106,94,539,110,225,93
5,Knak,94,146,15,35,540,96,344,45
6,Matti,54,36,214,123,595,63,27,92
7,Okholm,183,68,49,22,636,109,162,47
8,Tommy,189,94,27,88,414,138,219,65
9,Visti,56,106,77,132,425,110,193,83


,Manager,2025-01-28 00:00:00,2025-01-29 00:00:00,2025-01-30 00:00:00,2025-01-31 00:00:00,2025-02-01 00:00:00,2025-02-02 01:00:00,2025-02-06 00:00:00,2025-02-07 00:00:00,2025-02-08 00:00:00,...,2025-02-28 00:00:00,2025-03-01 11:00:00,2025-03-02 00:00:00,2025-03-02 11:00:00,2025-03-04 11:00:00,2025-03-05 11:00:00,2025-03-08 11:00:00,2025-03-09 00:00:00,2025-03-10 00:00:00,2025-03-11 00:00:00
0,Chrelle,3.0,21.0,2.0,11.0,86.0,NaN,1.0,15.0,6.0,...,3.0,NaN,16.0,18.0,8.0,10.0,5.0,NaN,2.0,1.0
1,Hustlersen,NaN,9.0,NaN,1.0,46.0,NaN,NaN,NaN,10.0,...,3.0,8.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2,Jappo,NaN,5.0,NaN,NaN,52.0,NaN,1.0,15.0,6.0,...,3.0,8.0,16.0,18.0,9.0,4.0,5.0,NaN,18.0,1.0
3,Jarma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,...,NaN,8.0,16.0,18.0,8.0,NaN,NaN,8.0,2.0,1.0
4,Kenk,NaN,5.0,NaN,NaN,40.0,1.0,NaN,NaN,NaN,...,3.0,8.0,16.0,22.0,8.0,4.0,5.0,14.0,14.0,NaN
5,Knak,NaN,4.0,NaN,NaN,12.0,NaN,NaN,NaN,6.0,...,NaN,41.0,NaN,18.0,8.0,22.0,NaN,NaN,NaN,1.0
6,Visti,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,55.0,16.0,22.0,5.0,NaN,NaN,14.0,12.0,NaN


In [15]:
rider_output.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
rider_output_cheapo.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)